# Robust Optimization in Portfolio Management

Author: Xiaochen (Lily) Wang, Chao (Kenneth) Wang

## Preliminary setting

In [1]:
using DataFrames, CSV # load data
using JuMP, Gurobi # modeling
using LinearAlgebra, Random, Printf, StatsBase, CategoricalArrays
using Plots, StatsPlots # plot
using Distributions

In [2]:
const GRB_ENV = Gurobi.Env(output_flag=0);

## Data

In [3]:
r = CSV.read("returns.csv", DataFrame, header=true);
r = r[:, 2:11];

In [4]:
# mean return of each stock
mu = [mean(c) for c in eachcol(r)];

# covariance matrix
function covmat(df)
    nc = ncol(df)
    t = zeros(nc, nc)
    for (i, c1) in enumerate(eachcol(df))
        for (j, c2) in enumerate(eachcol(df))
            sx, sy = skipmissings(c1, c2)
            t[i, j] = cov(collect(sx), collect(sy))
        end
    end
    return t
    end;

sigma = covmat(r);

In [5]:
sigma

10×10 Matrix{Float64}:
 0.00106454   0.000241516  0.000312346  …  1.37765e-5   0.000210137
 0.000241516  0.000377439  0.000201223     6.5931e-5    0.000185363
 0.000312346  0.000201223  0.000627007     3.53197e-5   0.000331141
 0.000305687  0.000139381  0.00018903      5.43634e-5   0.000158822
 0.000764281  0.000207765  0.000260861     2.6253e-5    0.000188921
 1.95074e-5   5.39378e-5   5.04883e-5   …  7.28555e-5   5.64445e-5
 0.000265883  0.000122608  0.000171051     5.44263e-5   0.000138244
 0.000200715  0.000271309  0.000182953     5.84385e-5   0.000174502
 1.37765e-5   6.5931e-5    3.53197e-5      0.000284176  5.97214e-5
 0.000210137  0.000185363  0.000331141     5.97214e-5   0.000528417

In [6]:
sigma_inv = inv(sigma)

10×10 Matrix{Float64}:
  4722.26     -292.663    -265.501   …    174.263     274.316     130.49
  -292.663    6991.57     -291.937      -4882.38     -258.458    -291.674
  -265.501    -291.937    2739.69        -236.957     231.229   -1260.99
   -81.0516   -259.981    -119.95          37.7844    -86.2598   -304.484
 -5207.05     -203.838     -41.0953      -388.067    -156.145    -138.86
   -65.0225   -722.726    -249.697   …   -332.656   -2706.0      -444.835
   248.59       31.3553   -452.404       -270.397    -394.456     -88.9033
   174.263   -4882.38     -236.957       7645.09     -151.402    -482.031
   274.316    -258.458     231.229       -151.402    4396.87     -136.261
   130.49     -291.674   -1260.99        -482.031    -136.261    3119.61

## Model

In [274]:
required_return = 0.0004
w_threshold = 0.4;

### Baseline

In [275]:
function train_baseline()
    model_base = Model(() -> Gurobi.Optimizer(GRB_ENV))

    I = 10;
    
    # decision variable
    @variable(model_base, w[1:I]>=0);
    
    # objective
    @objective(model_base, Min, sum(sigma[i, j] * w[i] * w[j] for i=1:I, j=1:I));
    
    # constraint
    @constraint(model_base, sum(mu[i] * w[i] for i=1:I) >= required_return);
    @constraint(model_base, threshold_constraint[i in 1:I], w[i] <= w_threshold);
    @constraint(model_base, sum(w[i] for i = 1:I) == 1);
 
    optimize!(model_base)
    
    w_opt = value.(w)
    
    return sqrt(objective_value(model_base)), w_opt
end

train_baseline (generic function with 1 method)

In [276]:
risk_bl, weight_bl = train_baseline();

In [277]:
risk_bl

0.009801154197425544

In [278]:
weight_bl

10-element Vector{Float64}:
 0.0001956700069706852
 0.03222423789157364
 6.4041527476231725e-6
 5.833534427958041e-7
 0.00022350791393546134
 0.39999984545965533
 0.20028926839062028
 0.1436727189148497
 0.22287808994954542
 0.0005096739665449994

### Robust

In [295]:
function train_model(rho)
    model_robust = Model(() -> Gurobi.Optimizer(GRB_ENV))

    I = 10;
    
    # decision variable
    @variable(model_robust, w[1:I]>=0);
    
    # objective
    @objective(model_robust, Min, sum(sigma[i, j] * w[i] * w[j] for i=1:I, j=1:I));
    
    # constraint
    @constraint(model_robust, rho^2 * sum(sigma_inv[i, j] * w[i] * w[j] for i=1:I, j=1:I) 
        <= (sum(mu[i] * w[i] for i=1:I) - required_return)^2);
    @constraint(model_robust, sum(mu[i] * w[i] for i=1:I) - required_return >= 0);
    @constraint(model_robust, threshold_constraint[i in 1:I], w[i] <= w_threshold);
    @constraint(model_robust, sum(w[i] for i = 1:I) == 1);

    optimize!(model_robust)
    
    w_opt = value.(w)
    
    return sqrt(objective_value(model_robust)), w_opt
end

train_model (generic function with 1 method)

In [407]:
required_return = 0.0003
risk_ro, weight_ro = train_model(0.0006);

In [297]:
risk_ro

0.10023703592908981

In [298]:
weight_ro

10-element Vector{Float64}:
 0.08964992837001691
 0.20283475621191632
 0.04389979575642731
 0.01786773000921725
 0.07466721618530123
 0.07975663664170851
 0.05721274923664928
 0.183409281828304
 0.16073761524973096
 0.090756165670171

#### Get feasible $\rho$

In [316]:
# for required_return = 0:0.0001:0.0005
# for w_threshold = 0.3:0.1:1

rho_list = zeros(0)

# for rho = 0.0001:0.00001:0.001
for rho = 0.0001:0.00001:0.001
    try
        required_return = 0.0004
        risk, returns = train_model(rho)
        append!(rho_list, rho)
    catch
    end
end

rho_list

42-element Vector{Float64}:
 0.0001
 0.00016
 0.00017
 0.00019
 0.0002
 0.00021
 0.00022
 0.00023
 0.00024
 0.00025
 0.00026
 0.00027
 0.00028
 ⋮
 0.00057
 0.00058
 0.00059
 0.00064
 0.00065
 0.00067
 0.0007
 0.00083
 0.00084
 0.00085
 0.00089
 0.0009

In [317]:
n = length(rho_list)

risk_list = zeros(n,)
weight_list = zeros(n, 10)
return_list = zeros(n,)
for i = 1:n
    risk, weight = train_model(rho_list[i])
    risk_list[i] = risk
    weight_list[i, :] = weight
    return_list[i] = sum(weight[i] * mu[i] for i=1:10)
end

In [318]:
df = zeros(n, 3);

df[:, 1] = rho_list;
df[:, 2] = return_list;
df[:, 3] = risk_list;

using DelimitedFiles
writedlm("robust.csv",  df, ',')

### Feasibility

In [319]:
r_std = [sqrt(sigma[i, i]) for i = 1:10];

In [320]:
n_samples = 10000
r_uncertain_set = zeros(n_samples, 10);

for i = 1:10
    d = Normal(mu[i], r_std[i])
    r_uncertain_set[:, i] = rand(d, n_samples)
end

#### Baseline

In [321]:
feasible_count_bl = 0
feasible_count_ro = 0

for sample = 1:n_samples
    new_r = r_uncertain_set[sample, :]
   
    if sum(new_r .* weight_bl) >= required_return
       feasible_count_bl = feasible_count_bl + 1
    end
    
    if sum(new_r .* weight_ro) >= required_return
       feasible_count_ro = feasible_count_ro + 1
    end

end

println("baseline: ", feasible_count_bl/n_samples)
println("robust: ", feasible_count_ro/n_samples)

baseline: 0.4998
robust: 0.4968


##### Robust

In [322]:
feasible_count_list = zeros(n)

for rho_i = 1:n
    cur_weight = weight_list[rho_i, :]
    
    for sample = 1:n_samples
        new_r = r_uncertain_set[sample, :]

        if sum(new_r .* cur_weight) >= required_return
           feasible_count_list[rho_i] = feasible_count_list[rho_i] + 1
        end
        
    end
end

In [323]:
feasible_count_list

42-element Vector{Float64}:
 4998.0
 4983.0
 4989.0
 4995.0
 4988.0
 4985.0
 4995.0
 4975.0
 4983.0
 4996.0
 4999.0
 5002.0
 5008.0
    ⋮
 4998.0
 4988.0
 4995.0
 4987.0
 5000.0
 4995.0
 4994.0
 4988.0
 4988.0
 4987.0
 4984.0
 4983.0

In [324]:
print(findmax(feasible_count_list))
print(findmin(feasible_count_list))

(5010.0, 24)(4975.0, 8)

In [404]:
n_samples = 100
r_uncertain_set = zeros(n_samples, 10);

for i = 1:10
    # r_uncertain_set[:, i] = rand(Uniform(mu[i] - 2*r_std[i], mu[i] + 2*r_std[i]), n_samples)
    d = Normal(mu[i], r_std[i])
    r_uncertain_set[:, i] = rand(d, n_samples)
end

rhos = [rho_list[i] for i in [ 1,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 16, 17, 18, 19, 20,
21, 22, 25, 26, 27, 37, 38, 39, 40, 41]]

feasible_total_count = zeros(length(rhos), 2)
win_count = 0

for i = 1:length(rhos)
    feasible_count_bl = 0
    feasible_count_ro = 0
    risk_ro, weight_ro = train_model(rhos[i]);
    for sample = 1:n_samples
        new_r = r_uncertain_set[sample, :]
    
        if sum(new_r .* weight_bl) >= required_return
        feasible_count_bl = feasible_count_bl + 1
        end
        
        if sum(new_r .* weight_ro) >= required_return
        feasible_count_ro = feasible_count_ro + 1
        end
    end
    feasible_total_count[i, 1] = feasible_count_bl/n_samples
    feasible_total_count[i, 2] = feasible_count_ro/n_samples
    if feasible_count_bl/n_samples < feasible_count_ro/n_samples
        win_count += 1
    end
end

win_count/length(rhos)

1.0

In [405]:
feasible_total_count

27×2 Matrix{Float64}:
 0.48  0.59
 0.48  0.59
 0.48  0.58
 0.48  0.59
 0.48  0.58
 0.48  0.59
 0.48  0.59
 0.48  0.59
 0.48  0.59
 0.48  0.59
 0.48  0.59
 0.48  0.59
 0.48  0.58
 ⋮     
 0.48  0.6
 0.48  0.6
 0.48  0.6
 0.48  0.6
 0.48  0.58
 0.48  0.61
 0.48  0.61
 0.48  0.59
 0.48  0.57
 0.48  0.57
 0.48  0.57
 0.48  0.57

In [203]:
rho_list = zeros()

for i=0.0001:0.00001:0.0002
    risk, returns = train_model(beta, b, i)
    append!( rho_list, returns )
    print(i)
end

LoadError: UndefVarError: beta not defined

In [ ]:
# beta = 0.0004; # minimum target return
# b = 0.4; # maximum budget weight for one single stock
# rho = 0.0001;

# rho_list = zeros(0)

# for i=0.0001:0.00001:0.0002
#     risk, returns = train_model(beta, b, i)
#     append!( rho_list, returns )
#     print(i)
# end

In [ ]:
objective_value(model_base), sum(mu[i] * w_opt[i] for i in I)

In [ ]:
w_opt = value.(w)
w_opt